In [55]:
import os
os.chdir("..")

In [61]:
os.environ["AZURE_LANGUAGE_ENDPOINT"] = "https://datathon.cognitiveservices.azure.com/"
os.environ["AZURE_LANGUAGE_KEY"] = "abdeac0882324fb59b2cf327e84f783a"
os.environ["AZURE_TRANSLATION_ENDPOINT"] = "https://api.cognitive.microsofttranslator.com/"
os.environ["AZURE_TRANSLATION_KEY"] = "ed4486cc91334c13863a64530126be25"

In [16]:
DEBUG = True

In [25]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from typing import Any, Dict, List

In [60]:
from ner import sample_analyze_healthcare_entities

In [1]:
input_text_de = """Asthma bronchiale
Auslöser sind Blütenpollen oder Tierhaare genauso wie Kälte oder Belastung: Asthma bronchiale gehört zu den häufigsten chronischen Krankheiten in der Schweiz.
Asthma bronchiale ist eine chronische und entzündliche Erkrankung der Atemwege. Charakteristisch sind Atemnot, Hustenanfälle und pfeifende Atemgeräusche. Die Bronchien reagieren dabei auf verschiedene Reize wie Pollen oder Tierhaare, aber auch Kälte oder körperliche Belastung können asthmatische Beschwerden auslösen.
In der Schweiz ist ungefähr jedes 10. Kind von Asthma betroffen. Damit ist Asthma bei Kindern die häufigste chronische Krankheit überhaupt. Die Erkrankung bessert sich mit steigendem Alter jedoch spürbar, deshalb leidet nur noch jeder 14. Erwachsene an Asthma.
"""

In [41]:
tokenizer_de_en = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
model_de_en = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-de-en")

tokenizer_en_de = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
model_en_de = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de")

summarizer_tokenizer = AutoTokenizer.from_pretrained("T-Systems-onsite/mt5-small-sum-de-en-v2")
summarizer_model = AutoModelForSeq2SeqLM.from_pretrained("T-Systems-onsite/mt5-small-sum-de-en-v2")

In [31]:
def translate(text: str, to_english: bool = True) -> str:
    if to_english:
        tokenized_text = tokenizer_de_en.prepare_seq2seq_batch([text], return_tensors='pt')
        translation = model_de_en.generate(**tokenized_text, max_length=1024)
        translated_text = tokenizer_de_en.batch_decode(translation, skip_special_tokens=True)[0]
    else:
        tokenized_text = tokenizer_en_de.prepare_seq2seq_batch([text], return_tensors='pt')
        translation = model_en_de.generate(**tokenized_text, max_length=1024)
        translated_text = tokenizer_en_de.batch_decode(translation, skip_special_tokens=True)[0]
    return translated_text

In [36]:
def abstract_summary(text: str, is_english: bool = True) -> str:
    if is_english:
        pass
        summary = text
    else:
        tokenized_text = summarizer_tokenizer.prepare_seq2seq_batch([text],max_length=1024, return_tensors='pt')
        output = summarizer_model.generate(**tokenized_text, max_length=1024)
        summary = summarizer_tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    return summary

In [37]:
def extractive_summary(text: str, is_english: bool = True) -> List[str]:
    if is_english:
        pass
        summary = [text]
    else:
        pass
        summary = [text]
    return summary

In [62]:
def ner(text: str, translated_lang: str = 'en') -> Dict[str, Any]:
    return sample_analyze_healthcare_entities([text], return_docs=True, translated_lang=translated_lang)

## English pipeline

In [39]:
input_text_en = translate(input_text_de)
abstract_summary_en = abstract_summary(input_text_en)
extractive_summary_en = extractive_summary(input_text_en)
ner_en = ner(input_text_en)

## German pipeline

In [63]:
abstract_summary_de = abstract_summary(input_text_de, is_english = False)
extractive_summary_de = extractive_summary(input_text_de, is_english = False)
ner_de = ner(input_text_de, translated_lang = 'de')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Input text

In [43]:
print(input_text_de)

Asthma bronchiale
Auslöser sind Blütenpollen oder Tierhaare genauso wie Kälte oder Belastung: Asthma bronchiale gehört zu den häufigsten chronischen Krankheiten in der Schweiz.
Asthma bronchiale ist eine chronische und entzündliche Erkrankung der Atemwege. Charakteristisch sind Atemnot, Hustenanfälle und pfeifende Atemgeräusche. Die Bronchien reagieren dabei auf verschiedene Reize wie Pollen oder Tierhaare, aber auch Kälte oder körperliche Belastung können asthmatische Beschwerden auslösen.
In der Schweiz ist ungefähr jedes 10. Kind von Asthma betroffen. Damit ist Asthma bei Kindern die häufigste chronische Krankheit überhaupt. Die Erkrankung bessert sich mit steigendem Alter jedoch spürbar, deshalb leidet nur noch jeder 14. Erwachsene an Asthma.



In [79]:
print(ner_de)
print(ner_de[0].keys())
print('______')
print([(x.category, x.text) for x in ner_de['entities'][0]])

{'entities': [[CategorizedEntity(text=Schweiz, category=Location, subcategory=GPE, length=7, offset=168, confidence_score=1.0), CategorizedEntity(text=eine, category=Quantity, subcategory=Number, length=4, offset=199, confidence_score=0.8), CategorizedEntity(text=Schweiz, category=Location, subcategory=GPE, length=7, offset=503, confidence_score=1.0), CategorizedEntity(text=10., category=Quantity, subcategory=Ordinal, length=3, offset=530, confidence_score=0.8), CategorizedEntity(text=Kind, category=PersonType, subcategory=None, length=4, offset=534, confidence_score=0.6), CategorizedEntity(text=Kindern, category=PersonType, subcategory=None, length=7, offset=582, confidence_score=0.61), CategorizedEntity(text=14., category=Quantity, subcategory=Ordinal, length=3, offset=731, confidence_score=0.8), CategorizedEntity(text=Erwachsene, category=PersonType, subcategory=None, length=10, offset=735, confidence_score=0.65)]], 'entity_relations': []}


KeyError: 0

In [48]:
if DEBUG:
    print("Input text translated to english:", end="\n\n")
    print(input_text_en, end="\n\n")
    print("--------------------------------", end="\n\n")
    
    print("Abstract summary in english:", end="\n\n")
    print(abstract_summary_en, end="\n\n")
    print("--------------------------------", end="\n\n")
    
    print("Extractive summary in english:", end="\n\n")
    print(extractive_summary_en, end="\n\n")

Input text translated to english:

Asthma bronchial triggers are flower pollen or animal hair as well as cold or strain: Asthma bronchial is one of the most common chronic diseases in Switzerland. Asthma bronchial is a chronic and inflammatory disease of the airways. Characteristic are shortness of breath, cough attacks and whistling breathing sounds. The bronchial bronchial reactions to various stimuli such as pollen or animal hair, but also cold or physical stress can cause asthmatic complaints. In Switzerland, about every 10th child is affected by asthma. Thus, asthma is the most common chronic disease in children. However, the disease improves with age, so only every 14th adult suffers from asthma.

--------------------------------

Abstract summary in english:

Asthma bronchial triggers are flower pollen or animal hair as well as cold or strain: Asthma bronchial is one of the most common chronic diseases in Switzerland. Asthma bronchial is a chronic and inflammatory disease of the